In [0]:
%%sh


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- explain (analyze True, costs True) 
-- SELECT *
-- FROM 
-- (SELECT *, int.product_id as product_id_int, dna.product_id as product_id_dna
-- FROM public.in_app_dna_info_mapping int RIGHT JOIN dna_genre_id_product_mapping dna ON int.product_id = dna.product_id
-- WHERE int.product_id is NULL) t1
-- WHERE not (t1.product_id_dna > 30000000000000 and t1.product_id_dna < 40000000000000) limit 10;
-- 
-- SELECT count(*)
-- FROM public.in_app_dna_info_mapping int LEFT JOIN dna_genre_id_product_mapping dna ON int.product_id = dna.product_id
-- WHERE dna.product_id is NULL limit 3;
-- select count(*) from public.in_app_dna_info_mapping limit 3;
-- \d public.in_app_dna_info_mapping;
select * from public.in_app_dna_info_mapping limit 3;
-- select * from public.in_app_dna_info_mapping where company_id is null limit 3;
SELECT count(*) FROM (select distinct company_id from public.in_app_dna_info_mapping) t1 LIMIT 3;
EOF

In [0]:

df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-dna/unified/dna.dna_universal_company_office.v1/dimension/")
df.show(10, False, True)

In [0]:
%%sh 
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.25.198  -U app_bdp_usage_qa -d dna -p 6432 << EOF 
set search_path=public;
-- \d public.dna_universal_company_office;
-- SELECT * FROM public.dna_universal_company_office LIMIT 10;
SELECT count(distinct universal_company_id) FROM public.dna_universal_company_office LIMIT 10;
EOF

In [0]:

from applications.db_check_v1.common.db_check_utils import query_df, pg_settings
class DNAMappingDBData(object):
    _pg_dsn = pg_settings('DNA')
    def get(self, sql):
        return query_df(self._pg_dsn, sql)

def _compare_df(df1, df2, left_on, right_on):
    for diff_type in ["left", "right"]:
        diff_df = df1.merge(df2, indicator=True, how=diff_type, left_on=left_on, right_on=right_on)
        temp_df = diff_df.loc[diff_df["_merge"] != "both"]
        print temp_df.empty
        if not temp_df.empty:
            print diff_type, ":"
            print diff_df
    

hq_sql = """SELECT DISTINCT universal_company_id, country, state, city FROM public.dna_universal_company_office"""
hq_pd_df = DNAMappingDBData().get(hq_sql)
hq_pd_df['universal_company_id'] = hq_pd_df['universal_company_id'].astype(str)
# print hq_pd_df

in_sql = """SELECT DISTINCT company_id FROM public.in_app_dna_info_mapping"""
in_pd_df = DNAMappingDBData().get(in_sql)
in_pd_df['company_id'] = in_pd_df['company_id'].astype(str)
in_pd_df['company_id'] = [i[:-2] for i in in_pd_df['company_id']]
# print in_pd_df

# _compare_df(hq_pd_df, in_pd_df, left_on='universal_company_id', right_on='company_id')


diff_df = hq_pd_df.merge(in_pd_df, indicator=True, how='outer', left_on='universal_company_id', right_on='company_id')
left_only_df = diff_df.loc[diff_df['_merge'] == 'left_only']
right_only_df = diff_df.loc[diff_df['_merge'] == 'right_only']
left_only_df.reset_index(drop=True, inplace=True)
right_only_df.reset_index(drop=True, inplace=True)
print left_only_df
print right_only_df
    

In [0]:

both_df = diff_df.loc[diff_df['_merge'] == 'both']
print both_df


In [0]:

# left_only_df['country'].isnull() | left_only_df['state'].isnull() | 
country_is_null = left_only_df.loc[left_only_df['city'].isnull()]
print country_is_null

In [0]:

left_only_df